# Open AI मोडेलहरूलाई फाइन ट्युनिङ

यो नोटबुक Open AI को [Fine Tuning](https://platform.openai.com/docs/guides/fine-tuning?WT.mc_id=academic-105485-koreyst) डक्युमेन्टेसनमा दिइएको हालको मार्गदर्शनमा आधारित छ।

फाइन-ट्युनिङले तपाईंको एप्लिकेसनका लागि फाउन्डेसन मोडेलहरूको प्रदर्शन सुधार गर्छ, किनभने यसले सो विशेष प्रयोग केस वा परिदृश्यसँग सम्बन्धित थप डेटा र सन्दर्भसहित मोडेललाई पुन: प्रशिक्षण दिन्छ। ध्यान दिनुहोस्, _few shot learning_ र _retrieval augmented generation_ जस्ता प्रॉम्प्ट इन्जिनियरिङ प्रविधिहरूले तपाईंलाई प्रासंगिक डेटा सहित डिफल्ट प्रॉम्प्टलाई समृद्ध बनाउन अनुमति दिन्छन्, जसले गुणस्तर सुधार गर्न मद्दत गर्छ। तर, यी तरिकाहरू लक्षित फाउन्डेसन मोडेलको अधिकतम टोकन विन्डो साइजले सीमित छन्।

फाइन-ट्युनिङसँग, हामी मोडेललाई आवश्यक डेटा सहित प्रभावकारी रूपमा पुन: प्रशिक्षण गर्दैछौं (हामीले अधिकतम टोकन विन्डोमा अटाउनेभन्दा धेरै उदाहरणहरू प्रयोग गर्न सक्छौं) - र मोडेलको _कस्टम_ संस्करण डिप्लोय गर्दैछौं, जसलाई अब इनफरेन्स समयमा उदाहरणहरू चाहिँदैन। यसले हाम्रो प्रॉम्प्ट डिजाइनको प्रभावकारिता मात्र सुधार्दैन (हामीसँग टोकन विन्डो अन्य कुराका लागि प्रयोग गर्न बढी लचिलोपन हुन्छ) तर सम्भावित रूपमा लागत पनि घटाउँछ (इनफरेन्स समयमा मोडेलमा पठाउनुपर्ने टोकनहरूको संख्या घटाएर)।

फाइन ट्युनिङका ४ चरणहरू छन्:
1. प्रशिक्षणका लागि डेटा तयार गर्नुहोस् र अपलोड गर्नुहोस्।
1. फाइन-ट्युनिङको लागि प्रशिक्षण जॉब चलाउनुहोस्।
1. फाइन-ट्युन गरिएको मोडेलको मूल्याङ्कन गर्नुहोस् र गुणस्तरका लागि दोहोर्याउनुहोस्।
1. सन्तुष्ट भएपछि इनफरेन्सका लागि फाइन-ट्युन गरिएको मोडेल डिप्लोय गर्नुहोस्।

ध्यान दिनुहोस्, सबै फाउन्डेसन मोडेलहरूले फाइन-ट्युनिङ समर्थन गर्दैनन् - [OpenAI डक्युमेन्टेसनमा हेर्नुहोस्](https://platform.openai.com/docs/guides/fine-tuning/what-models-can-be-fine-tuned?WT.mc_id=academic-105485-koreyst) पछिल्लो जानकारीका लागि। तपाईंले पहिले फाइन-ट्युन गरिएको मोडेललाई पनि फाइन-ट्युन गर्न सक्नुहुन्छ। यस ट्युटोरियलमा, हामी `gpt-35-turbo` लाई फाइन-ट्युनिङका लागि लक्षित फाउन्डेसन मोडेलको रूपमा प्रयोग गर्नेछौं।

---


### चरण १.१: आफ्नो डाटासेट तयार गर्नुहोस्

हामी एउटा च्याटबोट बनाउँदैछौं जसले तपाईंलाई आवर्त सारणीका तत्वहरू बुझ्न मद्दत गर्छ, जसले कुनै तत्वबारे सोधिएको प्रश्नको जवाफ लयात्मक कविता (लिमरिक) मा दिन्छ। _यस_ साधारण ट्युटोरियलमा, हामी केवल केही उदाहरणहरू सहितको डाटासेट बनाउनेछौं, जसले अपेक्षित डेटा ढाँचामा प्रतिक्रिया देखाउँछ। वास्तविक प्रयोगमा, तपाईंले धेरै उदाहरणहरू भएको डाटासेट बनाउनुपर्ने हुन्छ। तपाईंले चाहनु भएको क्षेत्रको लागि खुला डाटासेट उपलब्ध भएमा, त्यसलाई पनि प्रयोग गरेर आफ्नो आवश्यकता अनुसार ढाँचा परिवर्तन गर्न सक्नुहुन्छ।

हामी `gpt-35-turbo` मा केन्द्रित छौं र एकपटकको जवाफ (chat completion) चाहन्छौं, त्यसैले हामी [यो सिफारिस गरिएको ढाँचा](https://platform.openai.com/docs/guides/fine-tuning/preparing-your-dataset?WT.mc_id=academic-105485-koreyst) प्रयोग गरेर उदाहरणहरू बनाउन सक्छौं, जुन OpenAI को च्याट कम्प्लिसन आवश्यकतासँग मेल खान्छ। यदि तपाईंलाई बहुपटकको संवादात्मक सामग्री चाहिन्छ भने, तपाईंले [बहुपटकको उदाहरण ढाँचा](https://platform.openai.com/docs/guides/fine-tuning/multi-turn-chat-examples?WT.mc_id=academic-105485-koreyst) प्रयोग गर्नुहुन्छ, जसमा `weight` नामक प्यारामिटर हुन्छ, जसले कुन सन्देशहरू fine-tuning मा प्रयोग गर्ने/नगर्ने भन्ने संकेत गर्छ।

हामी यहाँको ट्युटोरियलका लागि सजिलो एकपटकको ढाँचा प्रयोग गर्नेछौं। डाटा [jsonl ढाँचा](https://jsonlines.org/?WT.mc_id=academic-105485-koreyst) मा हुन्छ, जहाँ प्रत्येक लाइनमा एउटा JSON-फर्म्याट गरिएको वस्तु हुन्छ। तलको स्निपेटमा २ वटा उदाहरण देखाइएको छ - पूरा नमुना सेट (१० वटा उदाहरण) को लागि [training-data.jsonl](../../../../../18-fine-tuning/python/openai/training-data.jsonl) हेर्नुहोस्, जुन हामी fine-tuning ट्युटोरियलमा प्रयोग गर्नेछौं। **नोट:** प्रत्येक रेकर्ड _एकै लाइनमा_ हुनुपर्छ (सामान्य JSON फाइलमा जस्तो लाइनमा विभाजन नगर्नुहोस्)

```json
{ "messages": [{"role": "system", "content": "Elle is a factual chatbot that answers questions about elements in the periodic table with a limerick"}, {"role": "user", "content": "Tell me about Gallium"}, {"role": "assistant", "content": "Gallium, oh gallium, so light - Melts in your hand, oh what a sight - At 86 degrees - Its liquid with ease - And in semiconductors, it's out of sight"}]}
{ "messages": [{"role": "system", "content": "Elle is a factual chatbot that answers questions about elements in the periodic table with a limerick"}, {"role": "user", "content": "Tell me about Hydrogen"}, {"role": "assistant", "content": "Hydrogen, the first in the line - The lightest of all, so divine - It's in water, you see - And in stars, it's the key - The universe's most common sign"}]}
```

वास्तविक प्रयोगमा राम्रो नतिजाका लागि तपाईंलाई धेरै ठूलो उदाहरण सेट चाहिन्छ - यहाँ गुणस्तर र fine-tuning को समय/खर्चबीच सन्तुलन हुन्छ। हामी सानो सेट प्रयोग गर्दैछौं ताकि fine-tuning प्रक्रिया छिटो देखाउन सकियोस्। थप जटिल fine-tuning ट्युटोरियलका लागि [OpenAI Cookbook को यो उदाहरण](https://github.com/openai/openai-cookbook/blob/main/examples/How_to_finetune_chat_models.ipynb?WT.mc_id=academic-105485-koreyst) हेर्नुहोस्।


### चरण १.२ आफ्नो डाटासेट अपलोड गर्नुहोस्

Files API प्रयोग गरेर डेटा अपलोड गर्नुहोस् [यहाँ वर्णन गरिएको अनुसार](https://platform.openai.com/docs/guides/fine-tuning/upload-a-training-file)। ध्यान दिनुहोस्, यो कोड चलाउनका लागि तपाईंले पहिले यी चरणहरू पूरा गरिसक्नुपर्छ:
 - `openai` Python प्याकेज इन्स्टल गर्नुभएको छ (नयाँ सुविधाहरूका लागि संस्करण >=0.28.0 प्रयोग गर्नुहोस्)
 - `OPENAI_API_KEY` environment variable लाई आफ्नो OpenAI API key मा सेट गर्नुभएको छ
थप जानकारीका लागि, कोर्सका लागि दिइएको [सेटअप गाइड](./../../../00-course-setup/02-setup-local.md?WT.mc_id=academic-105485-koreyst) हेर्नुहोस्।

अब, आफ्नो स्थानीय JSONL फाइलबाट अपलोड गर्नको लागि फाइल बनाउन तलको कोड चलाउनुहोस्।


In [24]:
from openai import OpenAI
client = OpenAI()

ft_file = client.files.create(
  file=open("./training-data.jsonl", "rb"),
  purpose="fine-tune"
)

print(ft_file)
print("Training File ID: " + ft_file.id)

FileObject(id='file-JdAJcagdOTG6ACNlFWzuzmyV', bytes=4021, created_at=1715566183, filename='training-data.jsonl', object='file', purpose='fine-tune', status='processed', status_details=None)
Training File ID: file-JdAJcagdOTG6ACNlFWzuzmyV


### चरण २.१: SDK प्रयोग गरेर फाइन-ट्युनिङ् जॉब बनाउनुहोस्


In [25]:
from openai import OpenAI
client = OpenAI()

ft_filejob = client.fine_tuning.jobs.create(
  training_file=ft_file.id, 
  model="gpt-3.5-turbo"
)

print(ft_filejob)
print("Fine-tuning Job ID: " + ft_filejob.id)

FineTuningJob(id='ftjob-Usfb9RjasncaZ5Cjbuh1XSCh', created_at=1715566184, error=Error(code=None, message=None, param=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs='auto', batch_size='auto', learning_rate_multiplier='auto'), model='gpt-3.5-turbo-0125', object='fine_tuning.job', organization_id='org-EZ6ag0n0S6Zm8eV9BSWKmE6l', result_files=[], seed=830529052, status='validating_files', trained_tokens=None, training_file='file-JdAJcagdOTG6ACNlFWzuzmyV', validation_file=None, estimated_finish=None, integrations=[], user_provided_suffix=None)
Fine-tuning Job ID: ftjob-Usfb9RjasncaZ5Cjbuh1XSCh


### चरण २.२: कामको स्थिति जाँच गर्नुहोस्

यहाँ `client.fine_tuning.jobs` API प्रयोग गरेर गर्न सकिने केहि कुरा छन्:
- `client.fine_tuning.jobs.list(limit=<n>)` - पछिल्ला n वटा फाइन-ट्युनिङ कामहरूको सूची हेर्नुहोस्
- `client.fine_tuning.jobs.retrieve(<job_id>)` - कुनै विशेष फाइन-ट्युनिङ कामको विवरण प्राप्त गर्नुहोस्
- `client.fine_tuning.jobs.cancel(<job_id>)` - फाइन-ट्युनिङ काम रद्द गर्नुहोस्
- `client.fine_tuning.jobs.list_events(fine_tuning_job_id=<job_id>, limit=<b>)` - कामबाट n वटा इभेन्टहरूको सूची हेर्नुहोस्
- `client.fine_tuning.jobs.create(model="gpt-35-turbo", training_file="your-training-file.jsonl", ...)`

यस प्रक्रियाको पहिलो चरण _training file को मान्यता जाँच गर्नु_ हो, जसले डाटा सही ढाँचामा छ कि छैन भन्ने सुनिश्चित गर्छ।


In [26]:
from openai import OpenAI
client = OpenAI()

# List 10 fine-tuning jobs
client.fine_tuning.jobs.list(limit=10)

# Retrieve the state of a fine-tune
client.fine_tuning.jobs.retrieve(ft_filejob.id)

# List up to 10 events from a fine-tuning job
client.fine_tuning.jobs.list_events(fine_tuning_job_id=ft_filejob.id, limit=10)

SyncCursorPage[FineTuningJobEvent](data=[FineTuningJobEvent(id='ftevent-GkWiDgZmOsuv4q5cSTEGscY6', created_at=1715566184, level='info', message='Validating training file: file-JdAJcagdOTG6ACNlFWzuzmyV', object='fine_tuning.job.event', data={}, type='message'), FineTuningJobEvent(id='ftevent-3899xdVTO3LN7Q7LkKLMJUnb', created_at=1715566184, level='info', message='Created fine-tuning job: ftjob-Usfb9RjasncaZ5Cjbuh1XSCh', object='fine_tuning.job.event', data={}, type='message')], object='list', has_more=False)

In [30]:
# Once the training data is validated
# Track the job status to see if it is running and when it is complete
from openai import OpenAI
client = OpenAI()

response = client.fine_tuning.jobs.retrieve(ft_filejob.id)

print("Job ID:", response.id)
print("Status:", response.status)
print("Trained Tokens:", response.trained_tokens)

Job ID: ftjob-Usfb9RjasncaZ5Cjbuh1XSCh
Status: running
Trained Tokens: None


### चरण २.३: प्रगति अनुगमन गर्न घटनाहरू ट्र्याक गर्नुहोस्


In [44]:
# You can also track progress in a more granular way by checking for events
# Refresh this code till you get the `The job has successfully completed` message
response = client.fine_tuning.jobs.list_events(ft_filejob.id)

events = response.data
events.reverse()

for event in events:
    print(event.message)

Step 85/100: training loss=0.14
Step 86/100: training loss=0.00
Step 87/100: training loss=0.00
Step 88/100: training loss=0.07
Step 89/100: training loss=0.00
Step 90/100: training loss=0.00
Step 91/100: training loss=0.00
Step 92/100: training loss=0.00
Step 93/100: training loss=0.00
Step 94/100: training loss=0.00
Step 95/100: training loss=0.08
Step 96/100: training loss=0.05
Step 97/100: training loss=0.00
Step 98/100: training loss=0.00
Step 99/100: training loss=0.00
Step 100/100: training loss=0.00
Checkpoint created at step 80 with Snapshot ID: ft:gpt-3.5-turbo-0125:bitnbot::9OFWyyF2:ckpt-step-80
Checkpoint created at step 90 with Snapshot ID: ft:gpt-3.5-turbo-0125:bitnbot::9OFWyzhK:ckpt-step-90
New fine-tuned model created: ft:gpt-3.5-turbo-0125:bitnbot::9OFWzNjz
The job has successfully completed


### चरण २.४: OpenAI ड्यासबोर्डमा स्थिति हेर्नुहोस्


तपाईंले OpenAI वेबसाइटमा गएर _Fine-tuning_ सेक्सनमा हेरेर पनि स्थिति हेर्न सक्नुहुन्छ। यसले तपाईंलाई हालको कामको स्थिति देखाउँछ, साथै अघिल्ला कामहरूको इतिहास पनि ट्र्याक गर्न दिन्छ। यस स्क्रिनसटमा, तपाईंले देख्न सक्नुहुन्छ कि अघिल्लो कार्य असफल भयो, र दोस्रो प्रयास सफल भयो। सन्दर्भका लागि, पहिलो प्रयासमा JSON फाइलमा गलत ढाँचामा रेकर्डहरू थिए - जब यो समस्या समाधान गरियो, दोस्रो प्रयास सफलतापूर्वक सम्पन्न भयो र मोडेल प्रयोगका लागि उपलब्ध भयो।

![Fine-tuning job status](../../../../../translated_images/fine-tuned-model-status.563271727bf7bfba7e3f73a201f8712fae3cea1c08f7c7f12ca469c06d234122.ne.png)


तपाईंले दृश्य ड्यासबोर्डमा तल स्क्रोल गरेर स्थिति सन्देशहरू र मेट्रिक्सहरू पनि हेर्न सक्नुहुन्छ:

| सन्देशहरू | मेट्रिक्सहरू |
|:---|:---|
| ![सन्देशहरू](../../../../../translated_images/fine-tuned-messages-panel.4ed0c2da5ea1313b3a706a66f66bf5007c379cd9219cfb74cb30c0b04b90c4c8.ne.png) |  ![मेट्रिक्सहरू](../../../../../translated_images/fine-tuned-metrics-panel.700d7e4995a652299584ab181536a6cfb67691a897a518b6c7a2aa0a17f1a30d.ne.png)|


### चरण ३.१: आईडी प्राप्त गर्नुहोस् र कोडमा फाइन-ट्युन गरिएको मोडेल परीक्षण गर्नुहोस्


In [46]:
# Retrieve the identity of the fine-tuned model once ready
response = client.fine_tuning.jobs.retrieve(ft_filejob.id)
fine_tuned_model_id = response.fine_tuned_model
print("Fine-tuned Model ID:", fine_tuned_model_id)

Fine-tuned Model ID: ft:gpt-3.5-turbo-0125:bitnbot::9OFWzNjz


In [47]:
# You can then use that model to generate completions from the SDK as shown
# Or you can load that model into the OpenAI Playground (in the UI) to validate it from there.
from openai import OpenAI
client = OpenAI()

completion = client.chat.completions.create(
  model=fine_tuned_model_id,
  messages=[
    {"role": "system", "content": "You are Elle, a factual chatbot that answers questions about elements in the periodic table with a limerick"},
    {"role": "user", "content": "Tell me about Strontium"},
  ]
)
print(completion.choices[0].message)

ChatCompletionMessage(content="Strontium, a metal so bright - It's in fireworks, a dazzling sight - It's in bones, you see - And in tea, it's the key - It's the fortieth, so pure, that's the right", role='assistant', function_call=None, tool_calls=None)


### चरण ३.२: Playground मा Fine-Tuned Model लोड गर्नुहोस् र परीक्षण गर्नुहोस्

अब तपाईंले fine-tuned model दुई तरिकाले परीक्षण गर्न सक्नुहुन्छ। पहिलो, तपाईं Playground मा जानुहोस् र Models drop-down बाट आफ्नो नयाँ fine-tuned model चयन गर्न सक्नुहुन्छ। अर्को विकल्प भनेको Fine-tuning प्यानलमा देखिएको "Playground" विकल्प प्रयोग गर्नु हो (माथिको screenshot हेर्नुहोस्), जसले _comparitive_ view खोल्छ जहाँ foundation र fine-tuned model दुबै सँगै देखिन्छन् र छिटो मूल्याङ्कन गर्न सकिन्छ।

![Fine-tuning job status](../../../../../translated_images/fine-tuned-playground-compare.56e06f0ad8922016497d39ced3d84ea296eec89073503f2bf346ec9718f913b5.ne.png)

तपाईंले आफ्नो training data मा प्रयोग भएको system context राख्नुहोस् र परीक्षण प्रश्न दिनुहोस्। तपाईंले देख्नुहुनेछ कि दुबै पक्षमा एउटै context र प्रश्न अपडेट हुन्छ। तुलना चलाउनुहोस् र दुबैको output मा फरक देख्नुहुनेछ। _ध्यान दिनुहोस्, fine-tuned model ले तपाईंले उदाहरणमा दिएको format मा उत्तर दिन्छ भने foundation model ले केवल system prompt पछ्याउँछ।_

![Fine-tuning job status](../../../../../translated_images/fine-tuned-playground-launch.5a26495c983c6350c227e05700a47a89002d132949a56fa4ff37f266ebe997b2.ne.png)

तपाईंले देख्नुहुनेछ कि यो तुलना मा प्रत्येक model को token count र inference को समय पनि देखिन्छ। **यो उदाहरण केवल प्रक्रिया देखाउनको लागि बनाइएको छ, वास्तविक dataset वा scenario को प्रतिनिधित्व गर्दैन।** तपाईंले देख्न सक्नुहुन्छ कि दुबै नमूनामा token संख्या समान छ (system context र user prompt एउटै छन्) तर fine-tuned model ले inference गर्न बढी समय लिन्छ (custom model भएको कारणले)।

वास्तविक जीवनमा, तपाईंले यस्ता सानो उदाहरण प्रयोग गर्नुहुन्न, बरु वास्तविक data (जस्तै, ग्राहक सेवा को लागि product catalog) मा fine-tuning गर्नुहुन्छ जहाँ उत्तरको गुणस्तर स्पष्ट देखिन्छ। _त्यो_ अवस्थामा, foundation model बाट समान गुणस्तरको उत्तर पाउन थप custom prompt engineering गर्नुपर्छ, जसले token प्रयोग बढाउँछ र सम्भवतः inference को processing समय पनि बढाउँछ। _यसलाई प्रयास गर्न, OpenAI Cookbook मा fine-tuning का उदाहरण हेर्नुहोस्।_



---

**अस्वीकरण**:  
यो दस्तावेज़ AI अनुवाद सेवा [Co-op Translator](https://github.com/Azure/co-op-translator) प्रयोग गरी अनुवाद गरिएको हो। हामी शुद्धताको लागि प्रयास गर्छौं, तर कृपया ध्यान दिनुहोस् कि स्वचालित अनुवादमा त्रुटि वा अशुद्धता हुन सक्छ। मूल भाषा भएको मूल दस्तावेज़लाई नै आधिकारिक स्रोत मान्नुपर्छ। महत्वपूर्ण जानकारीको लागि, व्यावसायिक मानव अनुवाद सिफारिस गरिन्छ। यस अनुवादको प्रयोगबाट उत्पन्न हुने कुनै पनि गलतफहमी वा गलत व्याख्याका लागि हामी जिम्मेवार हुने छैनौं।
